In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import roc_auc_score

In [2]:
data=pd.read_csv("/Users/lalitsachan/Dropbox/DLV1/Download Data/Subscribers.csv")

In [3]:
data.shape

(300000, 51)

In [4]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V42,V43,V44,V45,V46,V47,V48,V49,V50,Subscribers
0,-0.211875,743952.92,743952.92,200000.0,743952.92,200000.000,200000.0,-0.069396,943952.92,0.00,...,1.034232e+06,1966.57,1990.82,1752.89,100000.0,147.0,1826.046667,28.0,2557.22,0
1,0.241488,11295310.87,8034290.99,32000.0,8034290.99,18000.000,14000.0,0.167411,7508345.76,1160925.13,...,5.665658e+07,49957.65,11104.54,17711.99,38000.0,19.0,26467.683330,19.0,12807.07,0
2,0.459032,20363.68,317922.94,1650000.0,317922.94,1650000.000,1650000.0,46.068404,1655168.31,0.00,...,2.654720e+04,453.96,9.64,58.83,45072.0,0.0,315.506667,0.0,32.12,1
3,0.000000,856120.91,856120.91,0.0,856120.91,45554.885,0.0,0.000000,856120.91,0.00,...,8.506973e+05,2045.09,2045.09,2011.19,45072.0,30.0,2045.090000,19.0,2029.79,0
4,8.270783,194485.64,412330.33,835000.0,412330.33,835000.000,835000.0,-0.272225,2241.64,78625.00,...,5.572559e+04,70.01,100.60,89.59,31080.0,0.0,294.580000,0.0,26.95,0


In [5]:
y=pd.get_dummies(data['Subscribers'])

In [6]:
y

,0,1
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0
5,1,0
6,1,0
7,1,0
8,0,1
9,1,0


In [7]:
x=data.drop(['Subscribers'],axis=1)

In [8]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,
                                               random_state=2)

In [10]:
# x_train

In [11]:
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

In [12]:
st=StandardScaler()

In [13]:
st.fit(x_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [14]:
x_train=pd.DataFrame(st.transform(x_train))
x_test=pd.DataFrame(st.transform(x_test))

In [15]:
x_train.shape

(240000, 50)

# simple logistic regression

In [16]:
weights = tf.Variable(tf.random.normal(shape=(50, 2), dtype=tf.float64))
biases  = tf.Variable(tf.random.normal(shape=(2,), dtype=tf.float64))

In [17]:
def logistic_regression(x):
    lr = tf.add(tf.matmul(x, weights), biases)
    return lr


def cross_entropy(y_true, y_pred):
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    return tf.reduce_mean(loss)

def accuracy(y_true, y_pred):
    y_true = tf.cast(tf.argmax(y_true, axis=1), dtype=tf.int32)
    preds = tf.cast(tf.argmax(y_pred, axis=1), dtype=tf.int32)
    preds = tf.equal(y_true, preds)
    return tf.reduce_mean(tf.cast(preds, dtype=tf.float32))

def roc_auc(y_true,x):
    y_true = tf.cast(tf.argmax(y_true, axis=1), dtype=tf.int32).numpy()
    y_pred=tf.nn.softmax(logistic_regression(x))[:,1]
    return(roc_auc_score(y_true,y_pred))

def grad(x, y):
    with tf.GradientTape() as tape:
        y_pred = logistic_regression(x)
        loss_val = cross_entropy(y, y_pred)
    return tape.gradient(loss_val, [weights, biases])

In [18]:
epochs=2000

In [24]:
y_true=tf.cast(tf.argmax(y_test.values,axis=1),dtype=tf.int32)


In [32]:
preds=tf.cast(tf.argmax(logistic_regression(x_test.values),axis=1),dtype=tf.int32)

In [36]:
tf.reduce_mean(tf.cast(tf.equal(y_true,preds),dtype=tf.float32))

<tf.Tensor: id=87, shape=(), dtype=float32, numpy=0.60125>

In [37]:
learning_rate=0.01
for epoch in range(epochs):
    rand_ind=np.random.choice(range(x_train.shape[0]),100)
    outputs=y_train.iloc[rand_ind,:].values
    inputs=x_train.iloc[rand_ind,:].values
    
    dw,db=grad(inputs,tf.cast(outputs,'float32'))
    
    weights.assign_sub(learning_rate*dw)
    biases.assign_sub(learning_rate*db)
    
    if epoch%50==0:
        print(accuracy(y_test.values,logistic_regression(x_test.values)).numpy(),roc_auc(y_test.values,x_test.values))
    
    

0.60176665 0.4332304912887725
0.62953335 0.4428156397485393
0.65725 0.4517859360803791
0.68685 0.4635383439973049
0.7139 0.4763607804788827
0.7366167 0.4888217520840601
0.7522 0.5021394768598669
0.7646667 0.5141496260028668
0.7755167 0.5265219570555445
0.7859167 0.5381208495153939
0.7934333 0.5481984630729698
0.80083334 0.55723717160843
0.8063167 0.5672012281955553
0.81056666 0.5748753950511449
0.8147333 0.5815022586079338
0.81745 0.5871911254115898
0.8185833 0.5914233254355993
0.8193333 0.5947994228941059
0.82015 0.5987885603596542
0.82021666 0.6006240033872418
0.82033336 0.6040015827796474
0.8202 0.6078164000773412
0.82035 0.6099088125921891
0.8203833 0.6127322786319574
0.8203167 0.6149463621205301
0.8204 0.6168280567251707
0.82051665 0.6183513831775115
0.8204167 0.6199565647420991
0.8204167 0.6205453262368167
0.8204 0.6214830699768445
0.8204833 0.6231883851409479
0.8205 0.6239022914109361
0.82046664 0.6247782892076784
0.82055 0.6251827057450402
0.8208 0.625819285739668
0.82063335 0.

# classfier with a hidden layer

In [46]:
weights_h = tf.Variable(tf.random.truncated_normal(shape=(50, 20), dtype=tf.float64))
biases_h  = tf.Variable(tf.random.truncated_normal(shape=(20,), dtype=tf.float64))

weights_o = tf.Variable(tf.random.truncated_normal(shape=(20, 2), dtype=tf.float64))
biases_o  = tf.Variable(tf.random.truncated_normal(shape=(2,), dtype=tf.float64))

In [47]:
def logistic_regression(x):
    h_o = tf.add(tf.matmul(x, weights_h), biases_h)
    h_o=tf.nn.relu(h_o)
    lr=tf.add(tf.matmul(h_o, weights_o), biases_o)
    return lr


def cross_entropy(y_true, y_pred):

    loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    return tf.reduce_mean(loss)


def accuracy(y_true, y_pred):
    y_true = tf.cast(tf.argmax(y_true, axis=1), dtype=tf.int32)
    preds = tf.cast(tf.argmax(y_pred, axis=1), dtype=tf.int32)
    preds = tf.equal(y_true, preds)
    return tf.reduce_mean(tf.cast(preds, dtype=tf.float32))

def roc_auc(y_true,x):
    y_true = tf.cast(tf.argmax(y_true, axis=1), dtype=tf.int32).numpy()
    y_pred=tf.nn.softmax(logistic_regression(x))[:,1]
    return(roc_auc_score(y_true,y_pred))

def grad(x, y):
    with tf.GradientTape() as tape:
        y_pred = logistic_regression(x)
        loss_val = cross_entropy(y, y_pred)
    return tape.gradient(loss_val, [weights_h, biases_h,weights_o, biases_o])

In [48]:
epochs=2000

In [49]:
learning_rate=0.01
for epoch in range(epochs):
    rand_ind=np.random.choice(range(x_train.shape[0]),100)
    outputs=y_train.iloc[rand_ind,:].values
    inputs=x_train.iloc[rand_ind,:].values
    
    dw_h,db_h,dw_o,db_o=grad(inputs,tf.cast(outputs,'float32'))
    
    weights_h.assign_sub(learning_rate*dw_h)
    biases_h.assign_sub(learning_rate*db_h)
    weights_o.assign_sub(learning_rate*dw_o)
    biases_o.assign_sub(learning_rate*db_o)
    
    if epoch%50==0:
        print(accuracy(y_test.values,logistic_regression(x_test.values)).numpy(),roc_auc(y_test.values,x_test.values))
    
    

0.4182 0.4940074529962804
0.66153336 0.5420660310826384
0.73401666 0.5602627120836926
0.74831665 0.5743913585134424
0.7682 0.5917408246448503
0.78195 0.5974419171589532
0.78575 0.6002773236588729
0.78325 0.6040887032997496
0.78815 0.6059641136059241
0.79656667 0.6081882169995749
0.7970667 0.6114793557835423
0.80065 0.6140047352643195
0.79695 0.614146103161715
0.8020667 0.6174772082473847
0.80793333 0.6216634898119808
0.81591666 0.6244591087800753
0.81005 0.621897225076527
0.8107 0.6230655086916587
0.8155 0.6255788441625196
0.8132667 0.6272085425992675
0.81123334 0.6270385803134986
0.82351667 0.630996210320873
0.8166 0.6297133908969472
0.80728334 0.6328734952685455
0.81336665 0.6339699759136994
0.81631666 0.6321686105276006
0.8168333 0.6360640471785545
0.8193 0.6353130593470118
0.8203 0.6375375798703758
0.8206 0.6396559833762068
0.8189 0.6395797941944906
0.8214667 0.6397293591303489
0.82315 0.6412754997450371
0.8226167 0.6424351887296842
0.8175833 0.6429290195121283
0.8214333 0.64463810